### Open CV 를 이용해서 이미지 유사도를 테스트 하는 소스 테스트
#### 잘 동작하지 않는 것 같다 ;;

In [2]:
import cv2
import numpy as np

# 비교하고자 하는 두 이미지 로드
img1 = cv2.imread('C:\workspace\project\python-workspace\images\issue_2023-03-21_102948_edit.png')
img2 = cv2.imread('C:\workspace\project\python-workspace\images\Screenshot_20230321_143042_KB.jpg')

# 이미지를 그레이스케일로 변환
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# 이미지의 특징점 검출
sift = cv2.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1,None)
kp2, des2 = sift.detectAndCompute(gray2,None)

# 특징점 매칭 수행
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# 매칭 결과 정렬
good = []
for m,n in matches:
    if m.distance < 0.5*n.distance:
        good.append([m])

# 일치하는 이미지 반환
if len(good)>10:
    src_pts = np.float32([ kp1[m[0].queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m[0].trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w = gray1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    cv2.imshow("Result", img2)
    cv2.waitKey(0)
else:
    print("Not enough matches are found - {}/{}".format(len(good),10))